In [1]:
import sys
import pandas as pd
import numpy as np
from time import time
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier, SGDRegressor, Perceptron
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVC, SVR
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from joblib import Memory
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from ml_metrics import quadratic_weighted_kappa

def eval_wrapper(yhat, y):  
    y = np.array(y)
    y = y.astype(int)
    yhat = np.array(yhat)
    yhat = np.clip(np.round(yhat), np.min(y), np.max(y)).astype(int)   
    return quadratic_weighted_kappa(yhat, y)

#TODO: check which ones are really worth encoding and which can be even dropped (some may contain nonoverlapping values between
#test and training sest)
categorical = {'Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 
               'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 
               'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 
               'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 
               'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 
               'Insurance_History_9', 'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 
               'Medical_History_4', 'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 
               'Medical_History_8', 'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 
               'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 
               'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 
               'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 
               'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 
               'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 
               'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 
#                'Medical_History_38', 
               'Medical_History_39', 'Medical_History_40', 'Medical_History_41','Medical_History_1', 
               'Medical_History_15', 'Medical_History_24', 'Medical_History_32'}

In [2]:
import logging
# create logger
logger = logging.getLogger("logging_tryout2")
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)

# create formatter
# formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s")
formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s",
                              "%Y-%m-%d %H:%M:%S")
# add formatter to ch
ch.setFormatter(formatter)
# logger.addHandler(ch)
def info(msg):
    logger.info(msg[:120])

In [40]:
logging.basicConfig?

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
total = pd.concat([train, test])
median = total.median()
train.fillna(median, inplace=True)
test = test.fillna(median, inplace=True)
encoder = LabelEncoder()
for f in categorical:
    encoder.fit(total[f])
    train[f] = encoder.transform(train[f])
    test[f] = encoder.transform(test[f])

In [4]:
feature_cols = test.columns[1:]
categorical_inds = [i for i, col in enumerate(feature_cols) if col in categorical]
oh_encoder = OneHotEncoder(categorical_features=categorical_inds)

X = np.array(train[test.columns[1:]])
y = np.array(train.Response)
X_actual_test = np.array(test[feature_cols])

oh_encoder.fit(X)
X = oh_encoder.transform(X).todense()
X_actual_test = oh_encoder.transform(X_actual_test).todense()

train_test_folds = list(StratifiedKFold(y, n_folds=6, random_state=0))

In [5]:
train_cache = Memory(cachedir="cache/_train", verbose=0)
test_cache = Memory(cachedir="cache/test", verbose=0)

@train_cache.cache
def train_predictions(model):
    ind2pred = {}
    for i, (train, test) in enumerate(train_test_folds):
        info(("fitting fold   "+str(i+1)+ str(model)[:100]))
        model.fit(X[train], y[train])
        info(("fold fitted    "+str(i+1)+  str(model)[:100]))
        preds = model.predict(X[test])
        for i, p in zip(test, preds):
            ind2pred[i] = p
    
    return np.array([ind2pred[i] for i in range(len(y))])

@test_cache.cache
def test_predictions(model):
    model.fit(X, y)
    return model.predict(X_actual_test)


stacker_train_cache = Memory(cachedir="cache/stacker_train", verbose=0)
stacker_test_cache = Memory(cachedir="cache/stacker_test", verbose=0)

@stacker_train_cache.cache
def stacker_train_predictions(stacker, base_clfs):
    info("start stacker --------------------------")
    n = len(y)
    stacked_X = np.hstack([X] + [train_predictions(clf).reshape(n, 1) for clf in base_clfs])
    info("base regressors done")
    ind2pred = {}
    for i, (train, test) in enumerate(train_test_folds):
        info("fitting stacker fold %s   %s" % (i, str(stacker)))

        stacker.fit(stacked_X[train], y[train])
        info("stacker fitted fold %s    %s " % (i, str(stacker)))
        preds = stacker.predict(stacked_X[test])
        for i, p in zip(test, preds):
            ind2pred[i] = p
    info("stacker done =========================")
    return np.array([ind2pred[i] for i in range(len(y))])

@stacker_test_cache.cache
def stacker_test_predictions(stacker, base_clfs):
    n = len(y)
    stacked_X = np.hstack([X] + [train_predictions(clf).reshape(n, 1) for clf in base_clfs])
    stacker.fit(stacked_X, y)
    return stacker.predict(X_actual_test)

In [6]:
def benchmark(model):
    pred = train_predictions(model)
    return eval_wrapper(pred, y)

def make_predictions(model):
    model.fit(X, y)
    return model.predict(X_actual_test)

def benchmark_stacker(model, base_clfs):
    pred = stacker_train_predictions(model, base_clfs)
    return eval_wrapper(pred, y)


In [7]:
xgbr = XGBRegressor(objective="reg:linear", min_child_weight=80, subsample=0.85, colsample_bytree=0.30, silent=1, max_depth=9)
xgbc = XGBClassifier(objective="reg:linear", min_child_weight=80, subsample=0.85, colsample_bytree=0.30, silent=1, max_depth=9)
rfr = RandomForestRegressor(n_estimators=400)
etr = ExtraTreesRegressor(n_estimators=400)
sgdr = SGDRegressor()

In [8]:
start = time()
benchmark_stacker(xgbc, [xgbr])
end = time()
print end - start

2016-01-16 15:49:41;INFO;start stacker
INFO:logging_tryout2:start stacker
2016-01-16 15:49:41;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:51:52;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:51:52;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 15:54:03;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, co

6701.41893506


In [ ]:
benchmark_stacker(xgbc, [xgbr])

In [ ]:
%%time
benchmark_stacker(xgbc, [LinearRegression()])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="poly")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="rbf")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="sigmoid")])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [Perceptron()])

In [ ]:
%%time
benchmark_stacker(xgbc, [SVR(kernel="linear")])

In [ ]:
%%time
benchmark_stacker(xgbc, [rfrg])

In [ ]:
%%time
benchmark_stacker(xgbc, [rfre])

In [ ]:
%%time
benchmark_stacker(xgbc, [etrg])

In [ ]:
%%time
benchmark_stacker(xgbc, [etre])

In [ ]:
%%time
benchmark_stacker(xgbc, [sgdr])

In [9]:
benchmark_stacker(xgbc, [xgbr, rfr,  etr, sgdr, LinearRegression(), Perceptron(),
                                                 SVR(kernel="linear"), SVR(kernel="poly"), SVR(kernel="sigmoid"),
                                                 SVR(kernel="rbf")])

NameError: name 'rfre' is not defined

In [ ]:
super_predictions = stacker_test_predictions(xgbc[xgbr, rfre, rfrg, etre, etrg, sgdr, LinearRegression(), Perceptron(),
                                                 SVR(kernel="linear"), SVR(kernel="poly"), SVR(kernel="sigmoid"),
                                                 SVR(kernel="rbf")])

In [ ]:
def make_sub(stacker, base_clfs, filename):
    preds = stacker_test_predictions(stacker, base_clfs)
    
    df = pd.DataFrame()
    df['Id'] = test.Id
    df['Response'] = preds

    df.to_csv(filename, index=False)

In [ ]:
ExtraTreesClassifier(n_estimators=400)

In [43]:
small_X = X[:100, :]
small_y = y[:100]
model = LogisticRegression()
model.fit(small_X, small_y)
model.predict(small_X)

array([8, 4, 8, 8, 8, 8, 8, 1, 8, 1, 6, 2, 7, 3, 8, 5, 8, 7, 2, 8, 8, 5, 5,
       8, 6, 8, 6, 8, 7, 6, 4, 7, 7, 5, 2, 7, 8, 1, 1, 6, 6, 2, 5, 7, 2, 7,
       6, 8, 5, 2, 2, 7, 6, 4, 7, 8, 2, 7, 8, 6, 8, 8, 3, 5, 8, 6, 8, 7, 7,
       6, 6, 8, 5, 7, 6, 7, 6, 6, 6, 5, 4, 1, 7, 7, 1, 8, 8, 2, 6, 8, 8, 5,
       1, 8, 8, 6, 6, 1, 8, 8])

In [ ]:
regressor = XGBRegressor(objective="reg:linear", min_child_weight=80, subsample=0.85, colsample_bytree=0.30, silent=1, max_depth=9)
regressor.fit(X, y)
preds = regressor.predict(X)

In [ ]:
classifier = XGBClassifier()
classifier.fit(preds.reshape(len(y), 1), y)

In [ ]:
better_preds = classifier.predict(preds.reshape(len(y), 1))

In [ ]:
eval_wrapper(better_preds, y)

array([8, 4, 8, ..., 8, 8, 7])

In [ ]:
%%bash 
rm -r cache

In [ ]:
X.shape

In [ ]:
%%time 
n= 100000
small_x = X[:n, :]
small_y = y[:n]

xgbr.fit(small_x, small_y)

In [ ]:
small_x

In [ ]:
RandomForestRegressor?

In [ ]:
logger.info("asdfsdf")

In [ ]:
info("sadfsadf")

In [13]:
sorted([xgbr, rfr,  etr, sgdr, LinearRegression(), Perceptron(),
        SVR(kernel="linear"), SVR(kernel="poly"), SVR(kernel="sigmoid"),
        SVR(kernel="rbf")])

[ExtraTreesRegressor(bootstrap=False, compute_importances=None,
           criterion='mse', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
           min_samples_split=2, n_estimators=400, n_jobs=1, oob_score=False,
           random_state=None, verbose=0),
 LinearRegression(copy_X=True, fit_intercept=True, normalize=False),
 Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
       n_iter=5, n_jobs=1, penalty=None, random_state=0, shuffle=False,
       verbose=0, warm_start=False),
 RandomForestRegressor(bootstrap=True, compute_importances=None,
            criterion='mse', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=400, n_jobs=1,
            oob_score=False, random_state=None, verbose=0),
 SGDRegressor(alpha=0.0001, epsilon=0.1, eta0=0.01, fit_intercept=True,
        l1_ratio=0.15, learning_ra

In [14]:
from sklearn.dummy import DummyClassifier, DummyRegressor

In [26]:
tp = train_predictions(xgbr)

2016-01-16 20:20:12;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 20:22:24;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 20:22:24;INFO;fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fitting fold   XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
2016-01-16 20:24:35;INFO;fold fitted    XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.3,
       gamma=0, learning_rat
INFO:logging_tryout2:fold

In [38]:
benchmark_stacker(DummyClassifier(), [DummyRegressor()])

2016-01-16 20:46:30;INFO;start stacker
INFO:logging_tryout2:start stacker
2016-01-16 20:46:30;INFO;fitting fold   DummyRegressor(constant=None, strategy='mean')
INFO:logging_tryout2:fitting fold   DummyRegressor(constant=None, strategy='mean')
2016-01-16 20:46:30;INFO;fold fitted    DummyRegressor(constant=None, strategy='mean')
INFO:logging_tryout2:fold fitted    DummyRegressor(constant=None, strategy='mean')
2016-01-16 20:46:30;INFO;fitting fold   DummyRegressor(constant=None, strategy='mean')
INFO:logging_tryout2:fitting fold   DummyRegressor(constant=None, strategy='mean')
2016-01-16 20:46:30;INFO;fold fitted    DummyRegressor(constant=None, strategy='mean')
INFO:logging_tryout2:fold fitted    DummyRegressor(constant=None, strategy='mean')
2016-01-16 20:46:30;INFO;fitting fold   DummyRegressor(constant=None, strategy='mean')
INFO:logging_tryout2:fitting fold   DummyRegressor(constant=None, strategy='mean')
2016-01-16 20:46:30;INFO;fold fitted    DummyRegressor(constant=None, strate

0.00012598896826332417

In [37]:
%%bash
ls

cache
download.sh
example.log
keras.ipynb
my_submission.csv
oldcache
regression_training.ipynb
sample_submission.csv
test.csv
train.csv
training.ipynb
train_logs.txt
train.py
xgb_offset_submission.csv
xgboost.ipynb
